In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from typing import List, Tuple

import numpy as np

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    _ = torch.tensor([1.0], device=device)

torch.manual_seed(0)
torch.cuda.manual_seed(0)

Using device: cuda
NVIDIA GeForce RTX 4090


In [ ]:
class PINN(nn.Module):
    def __init__(self, num_hidden_layers=2, hidden_dim=32):
        super(PINN, self).__init__()
        self.input = nn.Linear(3, hidden_dim, bias=False)
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_dim, hidden_dim, bias=False) for _ in range(num_hidden_layers)])
        self.output = nn.Linear(hidden_dim, 1, bias=False)
        self.activation = nn.Tanh()
        #self.activation = nn.ReLU()
        self.init_weights()

    def forward(self, x, y, z):
        coords = torch.stack((x, y, z), dim=1).to(x.device)
        X = self.input(coords)
        X = self.activation(X)
        for layer in self.hidden_layers:
            X = layer(X)
            X = self.activation(X)
        X = self.output(X)
        return X

    def init_weights(self):
        nn.init.xavier_uniform_(self.input.weight)
        #nn.init.zeros_(self.input.bias)
        for layer in self.hidden_layers:
            nn.init.xavier_uniform_(layer.weight)
            #nn.init.zeros_(layer.bias)
        nn.init.xavier_uniform_(self.output.weight)
        #nn.init.zeros_(self.output.bias)

    def compute_derivatives(self, x, y, z):
        x = x.requires_grad_(True)
        y = y.requires_grad_(True)
        z = z.requires_grad_(True)
        u = self.forward(x, y, z)
        u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_y = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_z = torch.autograd.grad(u, z, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
        u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), create_graph=True)[0]
        u_zz = torch.autograd.grad(u_z, z, grad_outputs=torch.ones_like(u_z), create_graph=True)[0]
        return u, u_x, u_y, u_z, u_xx, u_yy, u_zz

In [14]:
# define source and boundary functions
# lets make something simple
# u = x^2 + y^2 + z^2
# u_xx + u_yy + u_zz = 6
# therefore, g = 6 s.t. u_xx + u_yy + u_zz = g
# simplest B.C. is therefore u =  x^2 + y^2 + z^2
def g(x: torch.Tensor, y: torch.Tensor, z: torch.Tensor) -> torch.Tensor:
    return 3 * torch.ones_like(x) 

def h(x: torch.Tensor, y: torch.Tensor, z: torch.Tensor) -> torch.Tensor:
    return x**2 + y**2 + z**2
    # return torch.zeros_like(x) # u = 0 on boundary

def f(x: torch.Tensor, y: torch.Tensor, z: torch.Tensor) -> torch.Tensor:
    return x**2
    #return x**2 + y**2 + z**2
    # return torch.zeros_like(x) # ignore the exact solution for now

In [15]:
# generate the training data
def generate_training_points(n_interior: int = 1024, n_boundary: int = 1024) -> Tuple[torch.Tensor, ...]:
    # interior points
    coords_i = torch.rand(n_interior, 3) * 2 - 1
    coords_i = coords_i.to(device)

    # boundary points
    coords_b = torch.rand(n_boundary, 3) * 2 - 1
    dim_set = torch.randint(0, 3, (n_boundary,)) # (B,)
    val_set = torch.randint(0, 2, (n_boundary,)).float() * 2 - 1 # (B,)
    coords_b[torch.arange(n_boundary), dim_set] = val_set
    coords_b = coords_b.to(device)

    return coords_i, coords_b

In [16]:
# coords_i, coords_b = generate_training_points(n_interior=16, n_boundary=16)
# print('Interior points:', coords_i.data)
# print('Boundary points:', coords_b.data)

In [ ]:
epochs = 10000
output_every = 1000

n_interior = 4096
n_boundary = 4096

lr = 1e-3
lambda_b = 1.0
lambda_i = 1.0

torch.manual_seed(0)
torch.cuda.manual_seed(0)

model = PINN(num_hidden_layers=2, hidden_dim=16).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=100)
# use a cosine scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-6)

# training data
coords_i, coords_b = generate_training_points(n_interior, n_boundary)
coords_i = coords_i.to(device)
coords_b = coords_b.to(device)

x_i, y_i, z_i = coords_i[:, 0], coords_i[:, 1], coords_i[:, 2]
x_b, y_b, z_b = coords_b[:, 0], coords_b[:, 1], coords_b[:, 2]

# # validation data
# coords_i, coords_b = generate_training_points(n_interior, n_boundary)
# coords_i = coords_i.to(device)
# coords_b = coords_b.to(device)

# x_v_i, y_v_i, z_v_i = coords_i[:, 0], coords_i[:, 1], coords_i[:, 2]
# x_v_b, y_v_b, z_v_b = coords_b[:, 0], coords_b[:, 1], coords_b[:, 2]


# print('Interior points:')
# print('x_i:', x_i.data)
# print('y_i:', y_i.data)
# print('z_i:', z_i.data)
# print('Boundary points:')
# print('x_b:', x_b.data)
# print('y_b:', y_b.data)
# print('z_b:', z_b.data)

# # check everything is on the right device
# print('x_i device:', x_i.device)
# print('y_i device:', y_i.device)
# print('z_i device:', z_i.device)
# print('x_b device:', x_b.device)
# print('y_b device:', y_b.device)
# print('z_b device:', z_b.device)
# # check model is on the right device
# print('Model device:', next(model.parameters()).device)
# # check model is on the right device
# print('Model device:', model.input.weight.device)
# # check model is on the right device
# print('Model device:', model.hidden_layers[0].weight.device)
# # check model is on the right device
# print('Model device:', model.output.weight.device)
# # check model is on the right device


for epoch in range(epochs):
    optimizer.zero_grad()

    # u_predict, u_x, u_y, u_z, u_xx, u_yy, u_zz = model.compute_derivatives(x_i, y_i, z_i)
    # u_predict = model(x_i, y_i, z_i)

    # # interior points:
    # g_i = g(x_i, y_i, z_i)
    # laplacian = u_xx + u_yy + u_zz
    # loss_i = torch.mean((laplacian - g_i)**2) * lambda_i

    # # boundary points:
    # u_predict_b = model(x_b, y_b, z_b)
    # h_b = h(x_b, y_b, z_b)
    # loss_b = torch.sqrt(torch.mean((u_predict_b - h_b)**2)) * lambda_b

    # total loss
    # loss = loss_i + loss_b
    # loss = loss_b
    loss = torch.mean((model(x_i, y_i, z_i) - f(x_i, y_i, z_i))**2) * lambda_i

    # optimization step
    loss.backward()
    # clip gradients
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step(loss)
    if epoch % output_every == 0:
        print(f"Epoch {epoch}, Training Loss: {loss.item():.6f}")
        print(f"gradient norm: {model.input.weight.grad.norm()}")
        for name, param in model.named_parameters():
            if param.grad is not None:
                print(f"{name} grad norm: {param.grad.norm().item()}")
        
        print()
        # print(f"Epoch {epoch}, Training Loss: {loss.item():.6f}, "
        #           f"PDE Loss: {loss_i.item():.6f}, "
        #           f"BC Loss: {loss_b.item():.6f}")
        
        # # validation
        # with torch.no_grad():
        #     u_predict = model(x_v_i, y_v_i, z_v_i)
        #     u_exact = f(x_v_i, y_v_i, z_v_i)
        #     loss_v_i = torch.mean((u_predict - u_exact)**2) * lambda_i

        #     h_predict_b = model(x_v_b, y_v_b, z_v_b)
        #     h_exact_b = h(x_v_b, y_v_b, z_v_b)
        #     loss_v_b = torch.mean((h_predict_b - h_exact_b)**2) * lambda_b

        #     total_loss_v = loss_v_i + loss_v_b
        #     print(f"(u_model - u)**2: {loss_v_i.item():.6f}, "
        #           f"(h_model - h)**2: {loss_v_b.item():.6f}, "
        #           f"Total Validation Loss: {total_loss_v.item():.6f}")

        # print(f"Epoch {epoch}, Validation Loss: {loss.item():.6f}, "
        #           f"PDE Loss: {loss_i.item():.6f}, "
        #           f"BC Loss: {loss_b.item():.6f}")
        # print()



/root/miniconda3/envs/pt/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 0, Training Loss: 0.266347
gradient norm: 0.3483726680278778
input.weight grad norm: 0.3483726680278778
hidden_layers.0.weight grad norm: 0.4650894105434418
hidden_layers.1.weight grad norm: 0.4969950318336487
output.weight grad norm: 0.34685638546943665

Epoch 1000, Training Loss: 0.202031
gradient norm: 0.00028335335082374513
input.weight grad norm: 0.00028335335082374513
hidden_layers.0.weight grad norm: 0.00029684562468901277
hidden_layers.1.weight grad norm: 0.0002264185022795573
output.weight grad norm: 0.0001639864785829559

Epoch 2000, Training Loss: 0.201577
gradient norm: 0.00031365646282210946
input.weight grad norm: 0.00031365646282210946
hidden_layers.0.weight grad norm: 0.00028123738593421876
hidden_layers.1.weight grad norm: 0.00018164599896408617
output.weight grad norm: 0.00015513415564782917

Epoch 3000, Training Loss: 0.201085
gradient norm: 0.00014419657236430794
input.weight grad norm: 0.00014419657236430794
hidden_layers.0.weight grad norm: 0.000183518975973

In [18]:
x = [1,0,-1]
y = [0,0,0]
z = [0,0,0]
x = torch.tensor(x).float().to(device)
y = torch.tensor(y).float().to(device)
z = torch.tensor(z).float().to(device)
u = model(x, y, z).cpu().detach().numpy()
exact = f(x, y, z).cpu().detach().numpy()
boundary = h(x, y, z).cpu().detach().numpy()

print(f"u: {u}\n f: {exact} boundary {boundary}")


u: [[-0.05503023]
 [ 0.        ]
 [ 0.05503023]]
 f: [1. 0. 1.] boundary [1. 0. 1.]


In [ ]:
# lets take a look at the weights and bias values after training. what histogram should we expect? For a well trained model, we should expect the weights to be normally distributed and the biases to be uniformly distributed.
import matplotlib.pyplot as plt
import seaborn as sns

def plot_weights(model: nn.Module):
    weights = []
    biases = []
    for i, layer in enumerate(model.hidden_layers):
        weights.append(layer.weight.data.cpu().numpy())
        biases.append(layer.bias.data.cpu().numpy())
        print(f"layer {i}:")
        print(f"weights: min: {weights[i].min()}, max: {weights[i].max()}, mean: {weights[i].mean()}, std: {weights[i].std()}")
        print(f"biases: min: {biases[i].min()}, max: {biases[i].max()}, mean: {biases[i].mean()}, std: {biases[i].std()}")
    
    
plot_weights(model)
